In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import time


In [ ]:
# WebDriver 설정
targetSite = 'https://series.naver.com/novel/top100List.series'
driver = webdriver.Chrome()
driver.get(targetSite)

wait = WebDriverWait(driver, 10)

# 각 탭 (실시간, 일간, 주간, 월간) 크롤링
for i in range(1, 5, 1):  # 실시간, 일간, 주간, 월간
    driver.find_element(By.XPATH, f'//*[@id="container"]/div[1]/ul/li[{i}]/a').click()
    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div/ul/li[1]/div[2]/h3/a')))
    
    Titles = []  # 제목 저장할 리스트
    Genre = []  # 장르 저장할 리스트
    Author = []  # 작가 저장할 리스트
    Start = []  # 연재 시작일
    Last = []  # 마지막 연재일
    Count = []  # 연재수
    Good = []  # 추천수
    Point = []  # 평점
    Comment = []  # 댓글 수
    
    for p in range(1, 6, 1):
        if p != 1:  # 첫 페이지가 아닌 경우 페이지 이동
            driver.find_element(By.XPATH, f'//*[@id="content"]/div/div[2]/a[{p-1}]').click()
            wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div/ul/li[1]/div[2]/h3/a')))
        
        for j in range(1, 21, 1):
            driver.find_element(By.XPATH, f'//*[@id="content"]/div/ul/li[{j}]/div[2]/h3/a').click()
            wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/h2')))
            
            if "login" in driver.current_url or "adult" in driver.current_url:
                Titles.append("19금")
                Genre.append("19금")
                Author.append("19금")
                Point.append("19금")
                Start.append("19금")
                Last.append("19금")
                Count.append("19금")
                Good.append("19금")
                Comment.append("19금")
                driver.back()
                continue
            
            # 연재 시작일과 마지막 연재일 수집
            a = driver.find_element(By.XPATH, '//*[@id="content"]/table/thead/tr/th[1]/div')  # 최신순
            b = driver.find_element(By.XPATH, '//*[@id="content"]/table/thead/tr/th[2]/div')  # 1화부터
            
            if a.get_attribute('class') == 'on _changeTicketSortOrder(DESC)':  # 최신순 정렬
                wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em')))
                Last.append(driver.find_element(By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em').text)
                b.click()
                wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em')))
                Start.append(driver.find_element(By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em').text)
                driver.back()
            
            elif b.get_attribute('class') == 'on _changeTicketSortOrder(ASC)':  # 1화부터 정렬
                wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em')))
                Start.append(driver.find_element(By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em').text)
                a.click()
                wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em')))
                Last.append(driver.find_element(By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em').text)
                driver.back()
            
            try:
                Titles.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/h2').text)
                Good.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div[2]/ul/li[2]/div/a/em').text)
            except:
                Titles.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/h2').text)
                Good.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[2]/ul/li[2]/div/a/em').text)
            
            try:
                Point.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div[1]/em').text)
            except:
                Point.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/em').text)
            
            Genre.append(driver.find_element(By.XPATH, '//*[@id="content"]/ul[1]/li/ul/li[2]/span/a').text)
            Author.append(driver.find_element(By.XPATH, '//*[@id="content"]/ul[1]/li/ul/li[3]/a').text)
            Count.append(driver.find_element(By.XPATH, '//*[@id="content"]/h5/strong').text)
            
            wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="cbox_module"]/div/div[1]/span')))
            Comment.append(driver.find_element(By.XPATH, '//*[@id="cbox_module"]/div/div[1]/span').text)
            
            print(f'p:{p}. {j}. 제목: {Titles[j-1]}, 장르: {Genre[j-1]}, 작가: {Author[j-1]}, 시작일: {Start[j-1]}, 최신화: {Last[j-1]}, 연재수: {Count[j-1]}, 추천수: {Good[j-1]}, 평점: {Point[j-1]}, 댓글: {Comment[j-1]}')
            driver.back()
    
    df = pd.DataFrame({"제목": Titles, "장르": Genre, "작가": Author, "연재 시작일": Start, "마지막 연재일": Last, "회수": Count, "댓글수": Comment, "추천수": Good, "평점": Point})
    df.index = df.index + 1
    
    if i == 1:
        df.to_csv('./data/NSeries_Now.txt')
    elif i == 2:
        df.to_csv('./data/NSeries_Today.txt')
    elif i == 3:
        df.to_csv('./data/NSeries_Week.txt')
    else:
        df.to_csv('./data/NSeries_Month.txt')

driver.quit()

In [3]:
dfN=pd.read_csv('./data/NSeries_Now.txt', index_col=0)
dfT=pd.read_csv('./data/NSeries_Today.txt', index_col=0)
dfW=pd.read_csv('./data/NSeries_Week.txt', index_col=0)
dfM=pd.read_csv('./data/NSeries_Month.txt', index_col=0)

In [4]:
dfN

,제목,장르,작가,연재 시작일,마지막 연재일,회수,댓글수,추천수,평점
1,멜팅 슬로우 [독점],로맨스,이른꽃,(2023.02.07.),(2023.04.01.),92,"3,912",101,9.8
2,회귀자 생활백서(回歸者 生活白書),무협,넉울히,(2024.05.23.),(2024.06.23.),157,"1,524",12,9.2
3,국세청 망나니 [독점],현판,동면거북이,(2020.07.08.),(2022.01.06.),500,"16,103",327,9.7
4,전직용병 재벌서자 [독점],현판,꾼밤,(2023.10.24.),(2024.06.24.),284,"2,143",28,9.2
5,회귀 경찰의 리셋 라이프,현판,한길,(2024.06.24.),(2024.06.24.),1039,2,좋아요,8.2
...,...,...,...,...,...,...,...,...,...
76,파혼 후 역대급 투자 천재,현판,단골손놈,(2024.06.21.),(2024.06.23.),106,3,1,7.6
77,회귀자의 성공 투자법 [독점],현판,박성호,(2021.09.22.),(2024.06.24.),736,"19,036",144,8.4
78,변경의 죄수는 검성을 꿈꾼다,판타지,Cadilet,(2024.05.27.),(2024.06.23.),130,484,6,9.6
79,방출당한 투수가 금강불괴로 돌아옴,현판,단팥빵소년,(2024.06.05.),(2024.06.24.),147,392,5,8.6


In [5]:
dfT

,제목,장르,작가,연재 시작일,마지막 연재일,회수,댓글수,추천수,평점
1,전지적 독자 시점,현판,싱숑,(2018.05.21.),(2024.06.21.),765,"815,565","20,448",9.9
2,영업 천재가 되었다,현판,댄킴,(2021.01.05.),(2021.09.03.),265,"5,578",126,9.9
3,북부 대공의 미친 데릴사위,판타지,곰돌이는,(2024.04.21.),(2024.06.22.),179,"4,255",54,9.7
4,"교수님, 그 결혼 저랑 해요 [독점]",로맨스,루위,(2024.06.07.),(2024.06.07.),81,246,11,9.9
5,회귀자 생활백서(回歸者 生活白書),무협,넉울히,(2024.05.23.),(2024.06.23.),157,"1,524",12,9.2
...,...,...,...,...,...,...,...,...,...
76,별을 품은 소드마스터,판타지,Q10,(2021.06.29.),(2022.04.18.),285,"17,475",568,9.8
77,은퇴한 초인의 슬로우라이프,판타지,ROHRAN노란,(2023.11.22.),(2024.05.14.),225,"2,117",18,9.7
78,재벌을 넘어 귀족으로,현판,푸달,(2024.05.24.),(2024.05.24.),200,64,좋아요,9.2
79,변경의 죄수는 검성을 꿈꾼다,판타지,Cadilet,(2024.05.27.),(2024.06.23.),130,484,6,9.6


In [6]:
dfW

,제목,장르,작가,연재 시작일,마지막 연재일,회수,댓글수,추천수,평점
1,괴력 난신(怪力 亂神),무협,한중월야,(2021.11.19.),(2023.07.25.),527,"22,292",259,9.7
2,차라리 빌런으로 살겠다,현판,서인하,(2024.06.20.),(2024.06.24.),125,418,9,9.6
3,"교수님, 그 결혼 저랑 해요 [독점]",로맨스,루위,(2024.06.07.),(2024.06.07.),81,246,11,9.9
4,독종의 연애 [독점],로맨스,플라비,(2024.06.20.),(2024.06.20.),85,148,8,9.4
5,북부 대공,로판,투머치텐션,(2024.01.10.),(2024.01.10.),162,"1,246",33,9.7
...,...,...,...,...,...,...,...,...,...
76,응급 맞선,로맨스,강영주,(2024.05.23.),(2024.05.23.),83,110,2,9.8
77,귀향한 먼치킨의 여행방송,현판,배고픈신발,(2024.06.07.),(2024.06.23.),126,99,4,9.0
78,당문전,무협,제이허빈,(2023.07.12.),(2024.06.24.),436,"14,705",91,9.4
79,재벌을 넘어 귀족으로,현판,푸달,(2024.05.24.),(2024.05.24.),200,64,좋아요,9.2


In [7]:
dfM

,제목,장르,작가,연재 시작일,마지막 연재일,회수,댓글수,추천수,평점
1,김 대리는 아이돌이 싫어 [독점],현판,퇴사연습생,(2023.11.07.),(2024.06.24.),270,"16,930",233,9.9
2,병약한 대공비의 남편은 기억 상실 [선공개],로판,천혜랑,(2024.05.17.),(2024.05.17.),84,23,1,8.0
3,귀농했더니 국가급 거물 후배들이 몰려든다,판타지,아길론,(2024.04.04.),(2024.06.18.),164,"1,792",27,9.5
4,운명을 보는 회사원,현판,영완(映完),(2020.01.10.),(2023.06.26.),317,"16,239",863,9.8
5,먹방으로 성공하는 ‘전’ 공작부인 [독점],로판,S모씨,(2024.05.03.),(2024.05.03.),125,67,5,7.3
...,...,...,...,...,...,...,...,...,...
76,유혹은 교만하게 [선공개],로맨스,문유하,(2024.05.03.),(2024.05.03.),75,82,6,9.7
77,악당 가족이 독립을 반대한다 [독점],로판,이흰,(2022.07.03.),(2024.04.23.),292,"3,402",321,9.9
78,검술 명가의 대마법사 막내딸 [독점],로판,윤하월,(2023.01.30.),(2023.10.02.),214,"1,164",44,9.8
79,훔친 용이지만 반품될까요 [선공개],로판,Onmi,(2024.05.03.),(2024.05.03.),100,15,5,8.6
